In [1]:
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model='gpt-4o')

### 4 Memories
* conversation memory
* conversation memory with window size
* conversation memory with token window size
* conversation memory with summary

In [2]:
# conversation memory
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=model, memory=memory, verbose=True)

C:\Users\62552\AppData\Local\Temp\ipykernel_33800\349466928.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(llm=model, memory=memory, verbose=True)


In [3]:
conversation.predict(input='Hello, who are you')

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, who are you
AI:

> Finished chain.


"Hello! I'm an AI designed to assist with a wide range of topics and provide information. I'm here to chat and help with any questions you might have. How can I assist you today?"

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. ProxyError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /info (Caused by ProxyError(\'Unable to connect to proxy\', SSLError(SSLZeroReturnError(6, \'TLS/SSL connection has been closed (EOF) (_ssl.c:1131)\'))))"))\nContent-Length: None\nAPI Key: lsv2_********************************************7c')
Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. ProxyError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/batch (Caused by ProxyError('Unable to connect to proxy', SSLError(SSLZeroReturnError(6, 'TLS/SSL connection 

In [4]:
conversation.predict(input="What is 1+1?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, who are you
AI: Hello! I'm an AI designed to assist with a wide range of topics and provide information. I'm here to chat and help with any questions you might have. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


"1 + 1 equals 2! It's one of the simplest and most fundamental math equations. If you'd like, I can also dive into how addition works or why this is true—just let me know!"

In [6]:
# view conversation memory
print(memory.buffer)
print("===========")
print(memory.load_memory_variables({}))

Human: Hello, who are you
AI: Hello! I'm an AI designed to assist with a wide range of topics and provide information. I'm here to chat and help with any questions you might have. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2! It's one of the simplest and most fundamental math equations. If you'd like, I can also dive into how addition works or why this is true—just let me know!
{'history': "Human: Hello, who are you\nAI: Hello! I'm an AI designed to assist with a wide range of topics and provide information. I'm here to chat and help with any questions you might have. How can I assist you today?\nHuman: What is 1+1?\nAI: 1 + 1 equals 2! It's one of the simplest and most fundamental math equations. If you'd like, I can also dive into how addition works or why this is true—just let me know!"}


In [9]:
# conversation memory with window size
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.buffer

'Human: Not much, just hanging\nAI: Cool'

In [17]:
# conversation memory with token size
from langchain.memory import ConversationTokenBufferMemory

llm = ChatOpenAI(temperature=0.0)
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

In [18]:
print(memory.load_memory_variables({})['history'])

AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

llm = ChatOpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100) 
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

In [14]:
print(memory.load_memory_variables({})['history'])

System: The human greets the AI, and they engage in casual conversation. The human inquires about the day's schedule, and the AI outlines a series of appointments: an 8 am meeting with the product team requiring a PowerPoint presentation, time from 9 am to 12 pm to work on the LangChain project, a lunch at an Italian restaurant with a customer at noon, and a reminder to bring a laptop to demonstrate the latest LLM.
